In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W34`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W35`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W36`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W37`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W38`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W39`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W40`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W41`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W42`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W43`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W44`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W45`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
96386,2020-08-20 16:30:56+00:00,2020-08-20 16:34:01+00:00,Production,0003 of 0021,C4,None,None,KL5,Wednesday Assembly,Thursday Afternoon,iXUsr_Chang(8908),5,3.08,2020-35
96387,2020-08-20 16:55:55+00:00,2020-08-20 16:59:30+00:00,Production,0010 of 0021,C4,None,None,KL5,Wednesday Assembly,Thursday Afternoon,iXUsr_Chang(8908),5,3.58,2020-35
96388,2020-08-20 14:38:38+00:00,2020-08-20 14:38:48+00:00,Production,0001 of 0001,SF,None,None,KL19,Wednesday Assembly,Thursday Afternoon,iXUsr_Chang(8908),6,0.17,2020-35
96389,2020-08-20 14:28:31+00:00,2020-08-20 14:28:45+00:00,Production,0006 of 0006,Q2,None,None,KL19,Wednesday Assembly,Thursday Afternoon,iXUsr_Chang(8908),6,0.23,2020-35
96390,2020-08-20 14:26:05+00:00,2020-08-20 14:28:31+00:00,Production,0005 of 0006,Q2,None,None,KL19,Wednesday Assembly,Thursday Afternoon,iXUsr_Chang(8908),6,2.43,2020-35


In [5]:
data_in_use.shape

(96391, 14)